# Data Processing script v2 for the SWEML v2.0
This .ipynb script uses python module for processing predownloaded NASA ASO observations by Water Year, locating nearest SNOTEL sites, connecting SNOTEL obs with ASO obs, and add geospatial features to the ML training/testing/hindcast dataframes.

In [15]:
import os
import numpy as np
HOME = os.getcwd()

#If you get a proj.db error below, run the following and put the following into the terminal
import pyproj
# Get the PROJ data directory
proj_data_dir = pyproj.datadir.get_data_dir()
proj_db_path = proj_data_dir + "/proj.db"
os.environ['PROJ_LIB'] =pyproj.datadir.get_data_dir()
os.environ['PROJ_LIB']
#set multiprocessing limits
CPUS = len(os.sched_getaffinity(0))
CPUS = int((CPUS/2)-2)

#set home to the head of the SWEMLv2.0 directory
# HOME = os.chdir('..')
# HOME = os.getcwd()

#Add your module here
# from utils.ASOget import ASODataProcessing_v2
import utils.get_InSitu_obs as get_InSitu_obs
import utils.GeoDF as GeoDF 
import utils.Obs_to_DF as Obs_to_DF 
import utils.get_VIIRS_SCA as get_VIIRS_SCA
import utils.get_Precip as get_Precip
import utils.get_Seasonality as get_Seasonality
import utils.vegetation_processer as vegpro
import utils.sturm_processer as stpro
from utils import ASOget

%load_ext autoreload
%autoreload 2

#make SWEMLv2.0 modeling domain for western USA
WY_list = np.arange(2013,2025)
# WY_list = [2013]
output_res = 750 #desired spatial resolution in meters (m)
threshold = 10
print(f"The current session is using {WY_list} years, {output_res}m resolution, and {CPUS} CPUs")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The current session is using [2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024] years, 750m resolution, and 6 CPUs


In [22]:
# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
directory = "Raw_ASO_Data"

#Get ASO data, sometime sites will give error and break code, most times you can just rerun it using the data_processor sections below (e.g., comment out other parts
for WY in WY_list:
    #Convert ASO tifs to parquet
    print(f"Converting ASO images for WY: {WY}")
    folder_name = f"{WY}/{directory}"
    data_processor = ASOget.ASODataProcessing_v2() #note, 2019-5-1, 2019-06-11 seems to be bad, manually removed from SW region
    data_processor.convert_tiff_to_parquet_multiprocess(folder_name, output_res, WY) 

Converting ASO images for WY: 2013
Converting 6 files  WY2013  750m resolution


WY2013:   0%|          | 0/6 [00:00<?, ?it/s]

Verifying:   0%|          | 0/6 [00:00<?, ?it/s]

All 6 parquet files verified.
Converting ASO images for WY: 2014
Converting 18 files  WY2014  750m resolution


WY2014:   0%|          | 0/18 [00:00<?, ?it/s]

...

Verifying:   0%|          | 0/82 [00:00<?, ?it/s]

All 82 parquet files verified.
Converting ASO images for WY: 2024
Converting 71 files  WY2024  750m resolution


WY2024:   0%|          | 0/71 [00:00<?, ?it/s]

Verifying:   0%|          | 0/71 [00:00<?, ?it/s]

All 71 parquet files verified.


## Get Snotel and CDEC in situ observations
- Ideas - add nearest sites elevation, distance from cell, then can bypass sites with bad data. 

In [ ]:
# Only needed once. Other spatial resolutions can use the same data
#Get in situ observations

#make a list of dates to align with the ASO observations (they go as early as Jan-29 and as far out as the July-17)
years = np.arange(2013,2025,1)
start_month_day = '10-01'
end_month_day = '08-31'

# observations 
get_InSitu_obs.Get_Monitoring_Data_Threaded_Updated(years, start_month_day, end_month_day, WY = True)

#combine years
get_InSitu_obs.combine_dfs(years)

## Code for generating ML dataframe using nearest in situ monitoring sites

In [16]:
# GeoDF used to create a dataframe for ML model development. Its function is to connect in situ observations to gridded locations
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        #load snotel meta location data, use haversine function
        GeoDF.fetch_snotel_sites_for_cellids(WY, output_res) # Using known up to date sites

        # Get geophysical attributes for each site, need to see how to add output resolution
        gdf = GeoDF.GeoSpatial(WY, output_res)

        #use geodataframe with lat/long meta of all sites to determine slope, aspect, and elevation
        metadf = GeoDF.extract_terrain_data_threaded(gdf, WY, output_res)
    else:
        print(f"No ASO data for {WY}")

2013
Loading all Geospatial prediction/observation files and concatenating into one dataframe


NameError: name 'tqdm_notebook' is not defined

## Connect Snotel to each ASO obs


In [35]:
#Connect nearest snotel observations with ASO data, makes a parquet file for each date  -  test to see if this works - need to just load the SNOTEL file, not collect them as in the function
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        dates = []
        manual = False
        Obs_to_DF.Nearest_Snotel_2_obs_MultiProcess(WY, output_res, manual, dates) 
    else:
        print(f"No ASO data for {WY}")


2013
Connecting site observations with nearest monitoring network obs
Loading observations
Loading 750M resolution grids for 2013 region
Processing datetime component of SNOTEL observation dataframe
Loading 6 processed ASO observations for the 2013 at 750M resolution
There are 6 aso dates in snotel obs
There are 0 missing snotel obs
Connecting 6 timesteps of observations for 2013


100%|██████████| 6/6 [00:00<00:00, 16.86it/s]

Site processing complete, adding observtional data to 20130403 df...


  0%|          | 0/2092 [00:00<?, ?it/s]

Site processing complete, adding observtional data to 20130429 df...


...

Site processing complete, adding observtional data to 20130601 df...


  0%|          | 0/2092 [00:00<?, ?it/s]

Site processing complete, adding observtional data to 20130608 df...


  0%|          | 0/2092 [00:00<?, ?it/s]

/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/Obsdf/USCATB_20130403_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/Obsdf/USCATB_20130601_ObsDF.parquet/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/Obsdf/USCATB_20130429_ObsDF.parquet

/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/Obsdf/USCATB_20130608_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/Obsdf/USCATB_20130503_ObsDF.parquet
/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/Obsdf/USCATB_20130525_ObsDF.parquet
Job complete for connecting SNOTEL obs to sites/dates


In [36]:
#Connect cell ids with ASO obs and snotel obs to geospatial features
for WY in WY_list:
    #path = f"{HOME}/SWEMLv2.0/data/ASO/{region}/{output_res}M_SWE_parquet"
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(WY)
        GeoDF.add_geospatial_threaded(WY, output_res)
    else:
        print(f"No ASO data for {region}")

2013
Loading geospatial metadata for grids in 2013
Loading all available processed ASO observations for 2013 at 750M resolution
Concatenating 6 with geospatial data...


  0%|          | 0/6 [00:00<?, ?it/s]

Job complete for connecting obs with geospatial data, the files can be found in /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/data/TrainingDFs/2013/750M_Resolution/GeoObsDFs


### Get NASA VIIRS fraction snow covered area for each location 

* Make sure the code grabs all dates for each region, may have to run multiple times
* run until "No granules found for DATE, requesting data from NSIDC..." no longer occurs


In [43]:
#check to see if the VIIRS data is available locally, if not, get from CIROH AWS - I think all of this data is for the incorrect year...
#get_VIIRS_SCA.get_VIIRS_from_AWS()

#Connect VIIRS data to dataframes
for WY in WY_list:
    path = f"{HOME}/data/ASO/{WY}/{output_res}M_SWE_parquet"
    if os.path.isdir(path) == True:
        print(WY)
        get_VIIRS_SCA.augment_SCA_multiprocessing(WY, output_res, threshold)
    else:
        print(f"No ASO data for {WY}")

2015
Getting VIIRS fsca values for 27 timesteps of observations for 2015


20150403, 20150412, 20150531, 20150426, 20150403, 20150412, 







  0%|          | 0/27 [00:00<?, ?it/s]

dataprocessing VIIRS for 20150531 complete...
20150426, 
dataprocessing VIIRS for 20150412 complete...
20150531, 
dataprocessing VIIRS for 20150412 complete...
20150428, 
dataprocessing VIIRS for 20150403 complete...
20150428, 
dataprocessing VIIRS for 20150403 complete...
20150326, 
dataprocessing VIIRS for 20150426 complete...
20150503, 
dataprocessing VIIRS for 20150428 complete...
20150527, 
dataprocessing VIIRS for 20150503 complete...
20150609, 
dataprocessing VIIRS for 20150326 complete...
20150217, 
dataprocessing VIIRS for 20150527 complete...
20150305, 
dataprocessing VIIRS for 20150609 complete...
20150325, 
dataprocessing VIIRS for 20150428 complete...
20150403, 
dataprocessing VIIRS for 20150426 complete...
20150409, 
dataprocessing VIIRS for 20150531 complete...
20150415, 
dataprocessing VIIRS for 20150217 complete...
20150427, 
dataprocessing VIIRS for 20150305 complete...
20150528, 
dataprocessing VIIRS for 20150325 complete...
20150608, 
dataprocessing VIIRS for 201504

### Add seasonality metrics to the dataframe

In [44]:
WY_list = [2013]

In [45]:
for WY in WY_list:
    #process snotel sites to make "snow hydrograph features" to determine above/below average WY conditions
    get_Seasonality.seasonal_snotel()

    #get the Day of season metric for each dataframe
    get_Seasonality.add_Seasonality(WY, output_res, threshold)

Adding Day of Season, seasonal nearest monitoring site averages, and nearest monitoring site anomaly to averages to all 2013 dataframes...


  0%|          | 0/6 [00:00<?, ?it/s]

### Use Sturm's snow classification as features within model framework

In [14]:
#download sturm data
stpro.get_Sturm_data()

for WY in WY_list:
    print(WY)
    input_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    sturm_file = f"{HOME}/data/SnowClassification/SnowClass_NA_300m_10.0arcsec_2021_v01.0.tif" #https://nsidc.org/data/nsidc-0768/versions/1
    output_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Sturm_Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    stpro.process_sturm_data_for_files(input_directory, sturm_file, output_directory)

File downloaded successfully!
2013


CPLE_AppDefined in PROJ: internal_proj_create_from_database: /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/envs/SWEML_310_v2/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.


Sturm file bounds: BoundingBox(left=-180.0, bottom=0.0, right=-10.0, top=90.0)
Adding Sturm snow classificaction to ASO gridcells


  0%|          | 0/6 [00:00<?, ?it/s]

CPLE_AppDefined in PROJ: internal_proj_create_from_database: /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/envs/SWEML_310_v2/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.
CPLE_AppDefined in PROJ: internal_proj_create_from_database: /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/envs/SWEML_310_v2/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.
CPLE_AppDefined in PROJ: internal_proj_create_from_database: /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/envs/SWEML_310_v2/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.
CPLE_AppDefined in PROJ: internal_proj_create_from_database: /uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/envs/SWEML_310_v2/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MI

### Add vegetation data to the dataframe from the North American land Cover Management Systemoutput_path

This script needs to be multiprocessed, too slow

In [47]:
#get data
url = "http://www.cread_parquetiles/atlas_layers/1_terrestrial_ecosystems/1_01_0_land_cover_2020_30m/usa_land_cover_2020v2_30m_tif.zip"
output_path = f"{HOME}/data/LandCover/"
file = "usa_land_cover_2020v2_30m_tif.zip" 
vegpro.get_data(url, output_path, file)
#unzip the file is not already done
#vegpro.unzip_LC_data(output_path, file)
#output = 1000 

for WY in WY_list:
    print(WY)
    input_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Sturm_Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    vegetation_file = f"{HOME}/data/LandCover/usa_land_cover_2020v2_30m_tif/USA_NALCMS_landcover_2020v2_30m/data/USA_NALCMS_landcover_2020v2_30m.tif"
    output_directory = f"{HOME}/data/TrainingDFs/{WY}/{output_res}M_Resolution/Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/{threshold}_fSCA_Thresh"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    vegpro.process_vegetation_data_for_files(input_directory, vegetation_file, output_directory)

Landcover data already downloaded
2013
Adding Vegetation criteria to ASO grids


  0%|          | 0/6 [00:00<?, ?it/s]

In [49]:
test = pd.read_parquet('data/TrainingDFs/2013/750M_Resolution/Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/Vegetation_Sturm_Season_VIIRS_GeoObsdf_USCATB_20130403.parquet')
test.head()

,cell_id,Date,cen_lat,cen_lon,Elevation_m,Slope_Deg,Aspect_Deg,Aspect_W,Aspect_N,swe_m,...,ns_5_week_mean,ns_6_week_mean,ns_1_anomoly,ns_2_anomoly,ns_3_anomoly,ns_4_anomoly,ns_5_anomoly,ns_6_anomoly,sturm_value,vegetation_value
0,750M_38.176695_-119.602174,2013-04-03,38.176695,-119.602174,3117,49,164,-0.276,-0.961,0.5882,...,109.905952,29.116667,0.873499,0.000000,0.775006,0.504840,0.778848,0.103034,5,8
1,750M_38.17834_-119.593795,2013-04-03,38.178340,-119.593795,2933,41,152,-0.467,-0.884,0.5735,...,109.905952,29.116667,0.873499,0.000000,0.775006,0.504840,0.778848,0.103034,5,1
2,750M_38.179984_-119.585416,2013-04-03,38.179984,-119.585416,2893,27,191,0.185,-0.983,0.5008,...,109.905952,29.116667,0.873499,0.000000,0.775006,0.504840,0.778848,0.103034,5,1
3,750M_38.166919_-119.616804,2013-04-03,38.166919,-119.616804,3059,13,34,-0.563,0.826,0.7872,...,56.853667,29.116667,0.873499,0.775006,0.000000,0.778848,0.504840,0.103034,5,8
4,750M_38.168564_-119.608427,2013-04-03,38.168564,-119.608427,2906,34,115,-0.909,-0.417,0.7147,...,56.853667,29.116667,0.873499,0.775006,0.000000,0.778848,0.504840,0.103034,5,1


### Get Daymet, NLDAS, gridMET Precipitation for each cell

In [3]:
datasets = [# 'Daymet',
             'NLDAS',
            #  'gridMET'
           ]

for dataset in datasets:
    for WY in WY_list:
        path = f"{HOME}/../../SWEMLdata/ASO/{WY}/{output_res}M_SWE_parquet"

        if os.path.isdir(path) == True:
            print(WY)
            get_Precip.get_precip(WY, output_res, threshold, dataset)
        else:
            print(f"No ASO data for {WY}, {path}")

        #Connect precipitation to processed DFs
        get_Precip.Make_Precip_DF(WY, output_res, threshold, dataset)

2013
Water Year start date: 2012-10-01


Processing precip files:   0%|          | 0/6 [00:00<?, ?it/s]

Getting precipitation data for 2013-04-03 at USCATB


ParserError: Error tokenizing data. C error: Expected 4 fields in line 15, saw 5


### Get AORC Precip for each cell

In [ ]:
dataset = 'AORC'
for WY in WY_list:
    path = f"{HOME}/../../SWEMLdata/ASO/{WY}/{output_res}M_SWE_parquet"

    if os.path.isdir(path) == True:
        print(WY)
        get_Precip.get_aorc_precip(WY, output_res, threshold)
    else:
        print(f"No ASO data for {WY}, {path}")

    #Connect precipitation to processed DFs
    get_Precip.Make_Precip_DF(WY, output_res, threshold, dataset)

### Get PRISM Precip for each cell

In [ ]:
for WY in WY_list:
    # get_Precip.get_prism(WY)
    get_Precip.add_prism_df(WY,output_res,threshold)

## Next steps
* Explore why errors in precip sites above
* add in situ obs - seasonality based on the historical neareste x monitoring stations - like a historical average to-date swe value unit hydrograph based on the day of year? This will include a historical time of year of normal swe value and a swe value of year compared to normal
* albedo metric


In [ ]:
import pandas as pd

HOME = os.path.expanduser('~')
region = 'Southwest'
output_res = '300'

dfpath = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"

SWmeta = pd.read_parquet(f"{dfpath}/{region}_metadata.parquet")

import UpdateDataFrame

#need to update the topographic features for every dataframe
output_res = '300'
training_cats = ['Obsdf']
fSCA = '' #'20_fSCA_Thresh'


for training_cat in training_cats:
    print(training_cat)

    for region in region_list:
        print(region)
        dfpath = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"
        #file to be used to updated training DF
        updatefile = pd.read_parquet(f"{dfpath}/{region}_metadata.parquet")


        #Update Dataframe
        UpdateDataFrame.updateTrainingDF(region, output_res, training_cat, fSCA, updatefile)

trainfile = pd.read_parquet(f"{dfpath}/{training_cat}/{fSCA}/Sturm_Season_Precip_VIIRS_GeoObsDF_20150406.parquet")

import matplotlib.pyplot as plt
import geopandas as gpd

from mpl_toolkits.axes_grid1 import make_axes_locatable

def SpatialAnalysis(EvalDF):
    #Convert to a geopandas DF
    Pred_Geo = gpd.GeoDataFrame(EvalDF, geometry = gpd.points_from_xy(EvalDF.cen_lon, EvalDF.cen_lat))

    Pred_Geo.plot(column='Elevation_m',
                  legend=False,
                )
    
SpatialAnalysis(trainfile)

In [ ]:
#check for nans
years = np.arange(2013,2024)
# years = [2020,2021,2022]
cols = ['cen_lat', 'cen_lon', 'Elevation_m', 'Slope_Deg',
       'Aspect_Deg', 'swe_m', 'ns_1', 'ns_2', 'ns_3', 'ns_4', 'ns_5', 'ns_6',
       'VIIRS_SCA', 'hasSnow', 'DOS', 'WY_week', 'ns_1_week_mean',
       'ns_2_week_mean', 'ns_3_week_mean', 'ns_4_week_mean', 'ns_5_week_mean',
       'ns_6_week_mean', 'ns_1_anomoly', 'ns_2_anomoly', 'ns_3_anomoly',
       'ns_4_anomoly', 'ns_5_anomoly', 'ns_6_anomoly', 
        # 'sturm_value',       'vegetation_value', 'Daymet', 'NLDAS'
       ]

for year in years:
    
    files = [filename for filename in os.listdir(f"{HOME}/data/TrainingDFs/{year}/1000M_Resolution/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/")]
    # files = [filename for filename in os.listdir(f"{HOME}/../Johnson/SWEMLv2.0/data/TrainingDFs/{year}/1000M_Resolution/Sturm_Seasonality_PrecipVIIRSGeoObsDFs/10_fSCA_Thresh/")]

    for file in files:
        test = pd.read_parquet(f"{HOME}/data/TrainingDFs/{year}/1000M_Resolution/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/{file}")
    # test = test.drop(columns='Date')
    # test.reset_index().drop(columns='index').head()
    # test.columns
        for column in cols:
            # print(len(test[test[column]<1]))
            # if len(test[test[column]<1]) > 500:
                # print(column)
            if np.isnan(test[column]).any():
                print('nan',file, column)
            if np.isinf(test[column]).any():
                print('inf',file,column)

# test.drop(columns='Elevation_m')[test.drop(columns='Elevation_m')>250].shape

In [5]:
import pandas as pd
test = pd.read_parquet('data/TrainingDFs/2021/750M_Resolution/AORCgridMETNLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/10_fSCA_Thresh/AORC_gridMET_NLDAS_PrecipDaymet_Vegetation_Sturm_Season_VIIRS_GeoObsdf_Animas_20210419.parquet')
test.head()

,cell_id,Date,cen_lat,cen_lon,Elevation_m,Slope_Deg,Aspect_Deg,swe_m,ns_1,ns_2,...,ns_5_anomoly,ns_6_anomoly,sturm_value,vegetation_value,Daymet,Aspect_N,Aspect_W,NLDAS,gridMET,AORC
0,2021_750M_37.96_-107.574,2021-04-19,37.960,-107.574,3801,38,22,0.360718,22.352,48.006,...,0.644301,0.882671,1,10,60.6,0.375,-0.927,46.0,47.8,44.1
1,2021_750M_37.96_-107.567,2021-04-19,37.960,-107.567,3968,17,69,0.257807,22.352,48.006,...,0.644301,0.882671,1,10,60.6,0.934,-0.358,46.0,47.8,43.7
2,2021_750M_37.953_-107.58,2021-04-19,37.953,-107.580,3720,6,85,0.367517,22.352,48.006,...,0.644301,0.882671,1,10,55.7,0.996,-0.087,46.0,54.9,44.6
3,2021_750M_37.953_-107.574,2021-04-19,37.953,-107.574,3650,12,65,0.403377,22.352,48.006,...,0.644301,0.882671,1,10,53.6,0.906,-0.423,46.0,47.8,44.6
4,2021_750M_37.953_-107.567,2021-04-19,37.953,-107.567,3764,45,70,0.389281,22.352,48.006,...,0.644301,0.882671,1,10,53.6,0.940,-0.342,46.0,47.8,44.2
